#<font color='lightgreen'>**Requests com a API do GitHub**</font>

##<font color='lightgreen'>Extração dos Dados</font>


In [ ]:
import requests
import pandas as pd
from math import ceil

In [ ]:
class DadosRepositorios:

 #criação da classe

  def __init__(self, owner):
    self.owner =  owner
    self.api_base_url = 'https://api.github.com'
    self.access_token = '*****CODIGO OCULTADO PARA SEGURANÇA'
    self.headers = {'X-GitHub-Api-Version': '2022-11-28',
                    'Authorization': 'Bearer ' + self.access_token}

  #extração dos repositórios

  def lista_repositorios (self):
    repos_list = []

    for page_num in range(1,20):
      try:
          url = f'{self.api_base_url}/users/{self.owner}/repos?page={page_num}'
          response = requests.get(url, headers = self.headers)
          repos_list.append(response.json())
      except:
          repos_list.append(None)

    return repos_list

  def extrai_conteudo(self,repos_list):
    names = []
    languages = []
    for page in repos_list:
      for repo in page:
        try:
          names.append(repo['name'])
          languages.append(repo['language'])
        except:
          pass

    return names,languages

  #transforma em dataframe

  def transforma_lista(self):
    repositorios = self.lista_repositorios()
    nomes, linguagens = self.extrai_conteudo(repositorios)
    dados = pd.DataFrame()
    dados['repository name'] = nomes
    dados['language'] = linguagens

    return dados

In [ ]:
#teste
amazon_rep =  DadosRepositorios('amzn')

In [ ]:
amazon_ling  = amazon_rep.transforma_lista()

In [ ]:
amazon_ling.head()

,repository name,language
0,.github,None
1,ads-advanced-tools-docs,None
2,ads-pao-amznjs-gtm-template,Smarty
3,alexa-coho,JavaScript
4,alexa-skills-kit-js,None


In [ ]:
amazon_ling.shape

(145, 2)

##<font color='lightgreen'>Transformação dos Dados</font>

In [ ]:
#extrai os repositorios de cada item de uma lista

def pega_dados_api(lista_content):
  dicio = {}
  for content in lista_content:
    rep = DadosRepositorios(content)
    dicio[content] = rep.transforma_lista()
    dicio[content].to_csv(f'/content/linguagens_{content}.csv')
  return dicio

In [ ]:
lista = ['amzn','netflix','spotify','apple']
dicio = pega_dados_api(lista)

In [ ]:
dicio['amzn']

,repository name,language
0,.github,None
1,ads-advanced-tools-docs,None
2,ads-pao-amznjs-gtm-template,Smarty
3,alexa-coho,JavaScript
4,alexa-skills-kit-js,None
...,...,...
140,zeek-plugin-enip,Zeek
141,zeek-plugin-profinet,Zeek
142,zeek-plugin-s7comm,Zeek
143,zeek-plugin-tds,Zeek


##<font color='lightgreen'>Armazenamento dos Dados</font>

In [ ]:
import base64

In [ ]:
#acesso e criação do repositório

#criacao da classe

class AdicionaRepositorio:

    def __init__(self, username):

      self.username = username
      self.api_base_url = 'https://api.github.com'
      self.access_token = '***CÓDIGO OCULTADO PARA SEGURANÇA'
      self.headers = {'X-GitHub-Api-Version': '2022-11-28',
           'Authorization': 'Bearer ' + self.access_token}

    def cria_repo(self, nome_repo):

        data = {
                "name": nome_repo,
                "description": "Dados extraídos dos repositórios de algumas empresas através da bibilioteca Requests",
                "private": False
               }
        r = requests.post(f"{self.api_base_url}/user/repos",
                                    json=data, headers=self.headers)

        print(f'status_code criação do repositório: {r.status_code}')

    def add_arquivo(self, nome_repo, nome_arquivo, caminho_arquivo):

        with open(caminho_arquivo, "rb") as file:
            file_content = file.read()
        encoded_content = base64.b64encode(file_content)

        url = f"{self.api_base_url}/repos/{self.username}/{nome_repo}/contents/{nome_arquivo}"
        data = {
                  "message": "Adicionando um novo arquivo",
                  "content": encoded_content.decode("utf-8")
                }

        r = requests.put(url, json=data, headers=self.headers)
        print(f'status_code upload do arquivo: {r.status_code}')


In [ ]:
#fazendo a criação do depositorio

In [ ]:
rep = AdicionaRepositorio('rr-mochiccino')
nome_rep = 'requests-api-github'
rep.cria_repo(nome_rep)

status_code criação do repositório: 201


In [ ]:
#mandando os arquivos para o GitHub

rep.add_arquivo(nome_rep, 'linguagens_amzn.csv', '/content/linguagens_amzn.csv')
rep.add_arquivo(nome_rep, 'linguagens_spotify.csv', '/content/linguagens_spotify.csv')
rep.add_arquivo(nome_rep, 'linguagens_netflix.csv', '/content/linguagens_netflix.csv')
rep.add_arquivo(nome_rep, 'linguagens_apple.csv', '/content/linguagens_apple.csv')


status_code upload do arquivo: 201
status_code upload do arquivo: 201
status_code upload do arquivo: 201
status_code upload do arquivo: 201
